In [35]:
import torch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

%pylab inline
pd.set_option('display.expand_frame_repr', True)

Populating the interactive namespace from numpy and matplotlib


# 準備訓練的資料

In [116]:
path = '/data/examples/pm25'

In [117]:
train_path = os.path.join(path, 'PM25_train.csv')
test_path = os.path.join(path, 'PM25_test.csv')
submission = os.path.join(path, 'submission.csv')

In [118]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
submit = pd.read_csv(submission)

In [119]:
# drop duplicate
df_train = df_train.drop_duplicates(keep='first')
df_test = df_test.drop_duplicates(keep='first')
df_all = pd.concat([df_train, df_test])

# feature

df_all['hours'] = df_all['Time'].str.split(':').str.get(0)
df_all = df_all.drop(['Date', 'Time','lat','lon'], axis=1)
df_all_one_hot = pd.get_dummies(df_all)

train_data = df_all_one_hot[0:1094714]
test_data = df_all_one_hot[1094714:]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [120]:
#確認資料的維度
print(train_data.shape)
print(test_data.shape)

(1094714, 281)
(41223, 281)


In [123]:
#拆分feature and target
train_feature, train_label = train_data, train_data.pop('PM2.5')
test_data = test_data.drop('PM2.5', axis=1)

In [189]:
#將資料轉成tensor 型態
x = torch.from_numpy(np.array(train_feature))
y = torch.from_numpy(np.array(train_label))

test = torch.from_numpy(np.array(test_data))

In [193]:
cuda0 = torch.device('cuda:0')

In [196]:
#將tensor 轉成float GPU型態
#(很重要，沒轉的話會沒辦法在GPU上面跑)
x = x.float().to(cuda0)
y = y.float().to(cuda0)

y = y.unsqueeze(1)

test = test.float().to(cuda0)

# 定義網絡結構

In [221]:
class poly_model(torch.nn.Module):
    def __init__(self):
        super(poly_model, self).__init__()
        self.linear1 = torch.nn.Linear(280,140)
        self.linear2 = torch.nn.Linear(140,70)
        self.linear3 = torch.nn.Linear(70,35)
        self.linear4 = torch.nn.Linear(35,20)
        self.linear5 = torch.nn.Linear(20,10)
        self.linear6 = torch.nn.Linear(10,1)
        
    def forward(self, x):
        lin_1 = self.linear1(x)
        lin_2 = self.linear2(lin_1)
        lin_3 = self.linear3(lin_2)
        lin_4 = self.linear4(lin_3)
        lin_5 = self.linear5(lin_4)
        out = self.linear6(lin_5)
        return out

In [222]:
#初始化model，並將model 轉成 GPU 型態
if torch.cuda.is_available():
    model = poly_model().cuda()
else:
    model = poly_model()

In [223]:
#定義loss function and optimizer
criterian = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0.9)

# 模型訓練

In [224]:
#定義epoch 數目(訓練的次數)
epoch = 3000

for i in range(epoch):
    #將x的值帶入model中
    output = model(x)
    
    #將output計算loss
    loss = criterian(output, y)
    
    #先將gradient的參數初使化
    optimizer.zero_grad()

    #計算backpropogation
    loss.backward()

    #update gradient
    optimizer.step()
    
    #顯示目前進度
    if i%500==0:
        print('Loss: {:.6f} after {} epoch'.format(loss, i))

print('finish traning')

Loss: 1737.425659 after 0 epoch
Loss: 17.371624 after 500 epoch
Loss: 17.092161 after 1000 epoch
Loss: 16.944618 after 1500 epoch
Loss: 16.870733 after 2000 epoch
Loss: 16.834743 after 2500 epoch
finish traning


# 看結果

In [229]:
#可以看各層的weight
model.linear6.weight

Parameter containing:
tensor([[ 0.0209,  0.1346, -0.0284, -0.4269, -0.5581,  0.1911, -0.3128,
          0.1188, -0.2426,  0.2504]], device='cuda:0')

In [230]:
#看各層的bias
model.linear6.bias

Parameter containing:
tensor([ 0.1520], device='cuda:0')

In [231]:
#看loss的狀況
loss

tensor(16.8158, device='cuda:0')

In [234]:
#實際帶值下去預測
predict = model(test)

In [237]:
#將值轉成numpy的型態
#要先將tensor轉成cpu tensor再轉Numpy
predict_np = predict.cpu().detach().numpy()

#計算各device的pm2.5
df_test['pred_pm25'] = predict_np
df_predict = pd.pivot_table(df_test, values='pred_pm25', index='device_id', aggfunc=np.mean)
df_predict_sumit = df_predict.reset_index()
df_predict_sumit.to_csv('mid_submit_nn.csv', index=False)